# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [1]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random
import json

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=1000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [3]:
# iterate all datasets in the eval_data folder
runners = {}
total_eval = 0
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=0)
    total_eval += len(runners[dataset].eval_data)
    
print("Total eval examples:", total_eval)
with open(f'results_checkpoint.json', 'r') as f:
    results = json.load(f)
with open(f'narratives_checkpoint.json', 'r') as f:
    narratives = json.load(f)

eval_data\housing_1.json
Total number of examples: 35
Labeled training examples: 5
Labeled evaluation examples: 15
Unlabeled training examples: 5
Unlabeled evaluation examples: 10
Max optimal length: 13.6
---
eval_data\housing_2.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 7
Unlabeled training examples: 5
Unlabeled evaluation examples: 5
Max optimal length: 10.0
---
eval_data\housing_3.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 8
Unlabeled training examples: 5
Unlabeled evaluation examples: 4
Max optimal length: 7.6
---
eval_data\mushroom_1.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
Max optimal length: 11.0
---
eval_data\mushroom_2.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation ex

In [4]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

In [5]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)
    

In [6]:
checkpoint=1

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [6]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        scores = runner.run_basic_prompting_experiment(prompt=prompt, max_iters=5)
        update_results("basic_prompting", dataset, scores, {"prompt": prompt})
        narratives.append(scores[2])
        pretty_print(scores)
        print("--")
    print("DUMPING...")
    with open(f'results_checkpoint.json', 'w') as f:
        json.dump(results, f)
    with open(f'narratives_checkpoint.json', 'w') as f:
        json.dump(narratives, f)
    print("DUMPING COMPLETE")
    print("=====")

Dataset: housing_1.json
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
Total score: 13.776470588235293 (accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 2.7764705882352936)
--
Prompt: You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.
Total score: 13.847058823529412 (accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 2.8470588235294114)
--
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.


KeyboardInterrupt: 

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [ ]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(n_few_shot=i, prompt=prompts[0], max_iters=5)
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        narratives.append(scores[2])
        pretty_print(scores)
        print("--")
    with open(f'results_checkpoint.json', 'w') as f:
        json.dump(results, f)
    with open(f'narratives_checkpoint.json', 'w') as f:
        json.dump(narratives, f)
    print("=====")

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [8]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i, j in [[3, 1], [3, 3], [5, 1], [5, 3], [5, 5]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(n_labeled_few_shot=i, n_bootstrapped_few_shot=j, max_iters=5)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        narratives.append(scores[2])
        pretty_print(scores)
        print("--")
    with open(f'results_checkpoint.json', 'w') as f:
        json.dump(results, f)
    with open(f'narratives_checkpoint.json', 'w') as f:
        json.dump(narratives, f)

Dataset: housing_1.json
Few-shot n: 3, Bootstrapped n: 1


  0%|          | 0/10 [00:00<?, ?it/s]


Bootstrapped 1 full traces after 1 examples in round 2.


  0%|          | 0/10 [00:00<?, ?it/s]


Bootstrapped 1 full traces after 1 examples in round 2.


...

Bootstrapped 5 full traces after 1 examples in round 2.


  0%|          | 0/10 [00:00<?, ?it/s]


Bootstrapped 5 full traces after 1 examples in round 2.


  0%|          | 0/10 [00:00<?, ?it/s]


Bootstrapped 5 full traces after 1 examples in round 2.
Total score: 15.925 (accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 3.925)
--


In [9]:
result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df

,dataset,total score,accuracy,completeness,fluency,conciseness,prompt,n_few_shot,n_bootstrapped_few_shot
0,housing_1.json,13.396078,4.0,4.0,3.2,2.196078,You are helping users understand an ML model's...,NaN,NaN
1,housing_1.json,13.274510,4.0,4.0,3.0,2.274510,You are helping users who do not have experien...,NaN,NaN
2,housing_1.json,14.010458,4.0,4.0,3.2,2.810458,You are helping users understand an ML model's...,NaN,NaN
3,housing_2.json,11.080000,4.0,4.0,2.6,0.480000,You are helping users understand an ML model's...,NaN,NaN
4,housing_2.json,11.160000,4.0,4.0,2.2,0.960000,You are helping users who do not have experien...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
110,student_2.json,15.250000,4.0,3.6,4.0,3.650000,You are helping users understand an ML model's...,3.0,1.0
111,student_2.json,15.025000,3.2,4.0,4.0,3.825000,You are helping users understand an ML model's...,3.0,3.0
112,student_2.json,16.000000,4.0,4.0,4.0,4.000000,You are helping users understand an ML model's...,5.0,1.0
113,student_2.json,16.000000,4.0,4.0,4.0,4.000000,You are helping users understand an ML model's...,5.0,3.0
